# Getting security register list available for trading in brazilian stock exchange

**What ?** The securities register list includes all ticker symbols for stocks, options, forwards, and other instruments available for trading on the Brazilian stock exchange. In addition to the ticker symbols, this database contains useful information such as expiration dates for derivatives, asset segments and categories, strike prices and underlying assets for options, corporate event dates, and much more.


**Why ?** This information is useful for qualifying financial instruments to achieve more comprehensive analysis by applying aggregations based on specific characteristics, thus enhancing the insights gained.


**How ?** The last 20 days of securities list informations are available on the B3 website
[(see this link)](https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/). From there, it will be manually downloaded to a local temporary folder. This notebook demonstrates how to read the downloaded .csv file and apply data cleaning and transformation using the provided data glossary as a guide [(link here)](https://www.b3.com.br/data/files/52/74/1E/14/4BA6D8103152D4C8AC094EA8/Glossario%20InstrumentsConsolidatedFile%202024.pdf). Finally, the cleaned data is uploaded to a local SQLite database for further analysis.

<img src="https://lh3.googleusercontent.com/d/1myP0jielNb2yjM9n0h2WOl5uKelUKctg" alt="securities_list_img" width="400" align="center">


## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os
import re

import sqlite3
import requests
import zipfile

#### Search at local SQLite database what is the last available data uploaded 

In [66]:
conn = sqlite3.connect(os.getenv('MY_FINANCE_DB_PATH')+'/finance_database.db')
cursor = conn.cursor()
cursor.execute('''SELECT ReportDate 
                    FROM B3_securities_register 
                    GROUP BY ReportDate ''') # this table was previously created to hold the trade by trade data

rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]

df_dt = pd.DataFrame(rows, columns=columns)
conn.close()

df_dt['ReportDate'].sort_values(ascending = False).head(3)

21    2024-05-29
20    2024-05-28
19    2024-05-27
Name: ReportDate, dtype: object

####  Looking for new files manually downloaded from B3 website into a local folder

In [62]:
file_path = os.path.join('temp_files') # Define the file path within the subfolder
all_files = [file for file in os.listdir(file_path) if "InstrumentsConsolidatedFile" in file]
all_files

['InstrumentsConsolidatedFile_20240429_1.csv',
 'InstrumentsConsolidatedFile_20240430_1.csv',
 'InstrumentsConsolidatedFile_20240502_1.csv',
 'InstrumentsConsolidatedFile_20240503_1.csv',
 'InstrumentsConsolidatedFile_20240506_1.csv',
 'InstrumentsConsolidatedFile_20240507_1.csv',
 'InstrumentsConsolidatedFile_20240508_1.csv',
 'InstrumentsConsolidatedFile_20240509_1.csv',
 'InstrumentsConsolidatedFile_20240510_1.csv',
 'InstrumentsConsolidatedFile_20240513_1.csv',
 'InstrumentsConsolidatedFile_20240514_1.csv',
 'InstrumentsConsolidatedFile_20240515_1.csv',
 'InstrumentsConsolidatedFile_20240516_1.csv',
 'InstrumentsConsolidatedFile_20240517_1.csv',
 'InstrumentsConsolidatedFile_20240520_1.csv',
 'InstrumentsConsolidatedFile_20240521_1.csv',
 'InstrumentsConsolidatedFile_20240522_1.csv',
 'InstrumentsConsolidatedFile_20240523_1.csv',
 'InstrumentsConsolidatedFile_20240524_1.csv',
 'InstrumentsConsolidatedFile_20240527_1.csv',
 'InstrumentsConsolidatedFile_20240528_1.csv',
 'Instruments

#### Extract, Transform and Load dataset

In [63]:
# Reading each .csv manually downlodaded from B3
####################################################################################################
df_app = pd.DataFrame()

for files in all_files: #zip_files_with_paths[::-1][6:8]

    df_app = pd.read_csv(file_path+'/'+files,sep = ";", encoding = "ISO-8859-1",low_memory=False, skiprows = 1, dtype = 'str')

# Data cleaning: changing columns names and data types
####################################################################################################

    # renaming columns
    new_columns_names = {'RptDt':'ReportDate',
                        'TckrSymb':'TickerSymbol',
                        'Asst':'Asset',
                        'AsstDesc':'AssetDescription',
                        'SgmtNm':'SegmentName',
                        'MktNm':'MarketName',
                        'SctyCtgyNm':'SecurityCategoryName',
                        'XprtnDt':'ExpirationDate',
                        'XprtnCd':'ExpirationCode',
                        'TradgStartDt':'TradingStartDate',
                        'TradgEndDt':'TradingEndDate',
                        'BaseCd':'BaseCode',
                        'ConvsCritNm':'ConversionCriteriaName',
                        'MtrtyDtTrgtPt':'MaturityDateTargetPoint',
                        'ReqrdConvsInd':'RequiredConversionIndicator',
                        'ISIN':'ISIN',
                        'CFICd':'CFICode',
                        'DlvryNtceStartDt':'DeliveryNoticeStartDate',
                        'DlvryNtceEndDt':'DeliveryNoticeEndDate',
                        'OptnTp':'OptionType',
                        'CtrctMltplr':'ContractMultiplier',
                        'AsstQtnQty':'AssetQuotationQuantity',
                        'AllcnRndLot':'AllocationLotSize',
                        'TradgCcy':'TradingCurrency',
                        'DlvryTpNm':'DeliveryTypeName',
                        'WdrwlDays':'WithdrawalDays',
                        'WrkgDays':'WorkingDays',
                        'ClnrDays':'CalendarDays',
                        'RlvrBasePricNm':'RolloverBasePriceName',
                        'OpngFutrPosDay':'OpeningFuturePositionDay',
                        'SdTpCd1':'SideTypeCode1',
                        'UndrlygTckrSymb1':'UnderlyingTickerSymbol1',
                        'SdTpCd2':'SideTypeCode2',
                        'UndrlygTckrSymb2':'UnderlyingTickerSymbol2',
                        'PureGoldWght':'PureGoldWeight',
                        'ExrcPric':'ExercisePrice',
                        'OptnStyle':'OptionStyle',
                        'ValTpNm':'ValueTypeName',
                        'PrmUpfrntInd':'PremiumUpfrontIndicator',
                        'OpngPosLmtDt':'OpeningPositionLimitDate',
                        'DstrbtnId':'DistributionIdentification',
                        'PricFctr':'PriceFactor',
                        'DaysToSttlm':'DaysToSettlement',
                        'SrsTpNm':'SeriesTypeName',
                        'PrtcnFlg':'ProtectionFlag',
                        'AutomtcExrcInd':'AutomaticExerciseIndicator',
                        'SpcfctnCd':'SpecificationCode',
                        'CrpnNm':'CorporationName',
                        'CorpActnStartDt':'CorporateActionStartDate',
                        'CtdyTrtmntTpNm':'CustodyTreatmentTypeName',
                        'MktCptlstn':'MarketCapitalisation',
                        'CorpGovnLvlNm':'CorporateGovernanceLevelName'}
    
    df_app.rename(columns = new_columns_names, inplace = True )

# changing data type
    df_app['BaseCode']=df_app['BaseCode'].str.replace(',', '.').astype(float)
    df_app['MaturityDateTargetPoint']=df_app['MaturityDateTargetPoint'].str.replace(',', '.').astype(float)
    df_app['ContractMultiplier ']=df_app['ContractMultiplier'].str.replace(',', '.').astype(float)
    df_app['AssetQuotationQuantity']=df_app['AssetQuotationQuantity'].str.replace(',', '.').astype(float)
    df_app['AllocationLotSize']=df_app['AllocationLotSize'].str.replace(',', '.').astype(float)
    df_app['WithdrawalDays']=df_app['WithdrawalDays'].str.replace(',', '.').astype(float)
    df_app['WorkingDays']=df_app['WorkingDays'].str.replace(',', '.').astype(float)
    df_app['CalendarDays']=df_app['CalendarDays'].str.replace(',', '.').astype(float)
    df_app['OpeningFuturePositionDay']=df_app['OpeningFuturePositionDay'].str.replace(',', '.').astype(float)
    df_app['PureGoldWeight']=df_app['PureGoldWeight'].str.replace(',', '.').astype(float)
    df_app['ExercisePrice']=df_app['ExercisePrice'].str.replace(',', '.').astype(float)
    df_app['PriceFactor']=df_app['PriceFactor'].str.replace(',', '.').astype(float)
    df_app['DaysToSettlement']=df_app['DaysToSettlement'].str.replace(',', '.').astype(float)
    df_app['MarketCapitalisation']=df_app['MarketCapitalisation'].str.replace(',', '.').astype(float)


# Write the dataframe into the SQLite database
####################################################################################################
    conn = sqlite3.connect(os.getenv('MY_FINANCE_DB_PATH')+'/finance_database.db')

    df_app.to_sql('B3_securities_register',conn,if_exists='append',index=False)
    
    # printing files read over each iteraction
    print(files)
    del df_app 
    df_app = pd.DataFrame()

InstrumentsConsolidatedFile_20240429_1.csv
InstrumentsConsolidatedFile_20240430_1.csv
InstrumentsConsolidatedFile_20240502_1.csv
InstrumentsConsolidatedFile_20240503_1.csv
InstrumentsConsolidatedFile_20240506_1.csv
InstrumentsConsolidatedFile_20240507_1.csv
InstrumentsConsolidatedFile_20240508_1.csv
InstrumentsConsolidatedFile_20240509_1.csv
InstrumentsConsolidatedFile_20240510_1.csv
InstrumentsConsolidatedFile_20240513_1.csv
InstrumentsConsolidatedFile_20240514_1.csv
InstrumentsConsolidatedFile_20240515_1.csv
InstrumentsConsolidatedFile_20240516_1.csv
InstrumentsConsolidatedFile_20240517_1.csv
InstrumentsConsolidatedFile_20240520_1.csv
InstrumentsConsolidatedFile_20240521_1.csv
InstrumentsConsolidatedFile_20240522_1.csv
InstrumentsConsolidatedFile_20240523_1.csv
InstrumentsConsolidatedFile_20240524_1.csv
InstrumentsConsolidatedFile_20240527_1.csv
InstrumentsConsolidatedFile_20240528_1.csv
InstrumentsConsolidatedFile_20240529_1.csv


#### Reading a sample from the SQLite database

In [64]:
# Show a sample of the data
conn = sqlite3.connect(os.getenv('MY_FINANCE_DB_PATH')+'/finance_database.db')
cursor = conn.cursor()
cursor.execute('''SELECT *
                    FROM B3_securities_register
                    WHERE SegmentName = 'CASH'
                        AND SecurityCategoryName =  'SHARES'
                        AND ReportDate = '2024-05-28'
                        AND Asset = 'PETR' ''') # reading a specifically ticker and date as an exlaple
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]

df_sample = pd.DataFrame(rows, columns=columns)
conn.close()

df_sample.head()

,ReportDate,TickerSymbol,Asset,AssetDescription,SegmentName,MarketName,SecurityCategoryName,ExpirationDate,ExpirationCode,TradingStartDate,...,SeriesTypeName,ProtectionFlag,AutomaticExerciseIndicator,SpecificationCode,CorporationName,CorporateActionStartDate,CustodyTreatmentTypeName,MarketCapitalisation,CorporateGovernanceLevelName,ContractMultiplier
0,2024-05-28,PETR3,PETR,PETR,CASH,EQUITY-CASH,SHARES,None,None,2024-05-03,...,None,None,None,ON N2,PETROLEO BRASILEIRO S.A. PETROBRAS,9999-12-31,FUNGIBLE,7.442454e+09,NIVEL 2,None
1,2024-05-28,PETR4,PETR,PETR,CASH,EQUITY-CASH,SHARES,None,None,2024-05-03,...,None,None,None,PN N2,PETROLEO BRASILEIRO S.A. PETROBRAS,9999-12-31,FUNGIBLE,5.602043e+09,NIVEL 2,None
